In [18]:
import os
import time
import queue
import requests
import threading
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# combine each player in one game as a gamelog

In [22]:
tStart = time.time()
# for i in range(2007, 2019):
for i in range(2019, 2020):
    mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i)
    side = os.listdir(mypath)
    for j in side:
        mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + j
        team = os.listdir(mypath)
        for k in team:
            
            mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + j + '\\' + k
            df = pd.read_excel(mypath + '\\total.xlsx')
            d = df['Date']
            date = list(d.unique())
            dropcol = ['index', 'Rk', 'G', 'Age', 'GS', 'MP']
            df = df.drop(dropcol, axis=1)
            ac = pd.DataFrame(columns=df.columns)
            for l in date:
                df = pd.read_excel(mypath + '\\total.xlsx')
                df.sort_values(by='Date')
                df = df.loc[df['Date'] == l, ]
                df = df.drop(dropcol, axis=1)
                dftest = df.loc[df['Date'] == l, ]
                dfsingle = list(dftest.iloc[1,:5])
                dfdate = df.loc[df['Date'] == l, ]
                dfdate = dfdate.drop(['Date', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7'],axis=1)
                dfdate = list(dfdate.sum())
                dfall = dfsingle + dfdate
                dic = {}
                for m in range(len(dfall)):
                    dic[df.columns[m]] = dfall[m]
                ac = ac.append(dic, ignore_index=True)
            ac['Unnamed: 5'] = ac['Unnamed: 5'].fillna('away')
            ac['Unnamed: 5'] = ac['Unnamed: 5'].replace('@', 'home')
            alist = []
            for n in ac['Unnamed: 7']:
                a = ''
                for o in n:
                    if o == ' ':
                        break
                    else:
                        a = a + o
                alist.append(a)
            ac['Unnamed: 7'] = alist
            ac['Unnamed: 7'] = ac['Unnamed: 7'].replace('L', 0)
            ac['Unnamed: 7'] = ac['Unnamed: 7'].replace('W', 1)
            ac = ac.rename(columns={'Unnamed: 5': 'place', 'Unnamed: 7': 'result'})
            ac = ac[['Date', 'Tm', 'place', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA',
                   '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
                   'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'result']]
            path = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog' + '\\' + str(i) + '\\' + j + '\\' + k
            if not os.path.isdir(path):
                os.makedirs(path)
            ac.to_excel(path + '\\gamelog.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))

Cost 59.159528 s


# Pre-Processing data to home minus away in a game

In [23]:
tStart = time.time()
# for k in range(2007, 2019):
for k in range(2019, 2020):
    mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k)
    side = os.listdir(mypath)
    for l in side:
        mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\' + l
        team = os.listdir(mypath)
        for m in team:
            teamlog = []
            mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\' + l + '\\' + m
            df = pd.read_excel(mypath + '\\gamelog.xlsx')
            for i in range(len(df)):
                sli = []
                opponent = df.loc[i,][3]
                if opponent == 'NJN':
                    opponent = 'BRK'
                elif opponent == 'CHA':
                    opponent = 'CHO'
                elif opponent == 'NOK' or opponent == 'NOH':
                    opponent = 'NOP'
                elif opponent == 'SEA':
                    opponent = 'OKC'
                else:
                    opponent = opponent
                gamedate = df.loc[i,][0]
                count = 0
                for t in team:
                    if opponent != t:
                        continue
                    else:
                        count = count + 1
                if count > 0:
                    getpath = ('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\' + l + '\\' + opponent + '\\gamelog.xlsx')
                else:
                    if l == 'East':
                        getpath = ('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\West\\' + opponent + '\\gamelog.xlsx')
                    else:
                        getpath = ('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\East\\' + opponent + '\\gamelog.xlsx')
                dfo = pd.read_excel(getpath)
                dfone = list(dfo[dfo.Date == gamedate].loc[dfo[dfo.Date == gamedate].index.values[0]])
                for z in range(4):
                    sli.append(df.loc[i][z])
                for j in range(4,24):
                    a = df.loc[i][j] - dfone[j]
                    sli.append(a)
                sli.append(df.loc[i][24])
                teamlog.append(sli)
            df1 = pd.DataFrame(teamlog, columns=df.columns)
            df1.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(k) + '\\' + l + '\\' + m + '\\home_away.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))

Cost 7.673000 s


# delete file when get an error

In [2]:
for i in range(2007, 2019):
    mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i)
    side = os.listdir(mypath)
    for j in side:
        mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j
        team = os.listdir(mypath)
        for k in team:
            teamlog = []
            mypath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\' + str(i) + '\\' + j + '\\' + k
            if os.path.exists(mypath + '\\home_away.xlsx'):
                os.remove(mypath + '\\home_away.xlsx')
            else:
                print(mypath + '\\home_away.xlsx does not exist!!')

In [14]:
month = ['october.xlsx', 'november.xlsx', 'december.xlsx', 'january.xlsx', 'february.xlsx', 'march.xlsx', 'april.xlsx']
year = os.listdir('C:\\Users\\ecom\\Desktop\\NBAData\\schedual')
for i in year:
    mon = os.listdir('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i))
    for j in mon:
        for k in month:
            if j == k:
                os.remove('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '\\' + j)

In [13]:
df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\2019\\october.xlsx')
for i in range(2017, 2020):
    df1 = pd.DataFrame(columns=df.columns)
    month = os.listdir('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i))
    for j in month:
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '\\' + j
        dfcon = pd.read_excel(path)
        df1 = df1.append(dfcon, ignore_index=True)
    df1 = df1.sort_values(by='Date')
    df1.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '\\' + str(i) + '_season_schedual.xlsx')
    print(str(i) + 'is completed')
print('Done')

2017is completed
2018is completed
2019is completed
Done


In [17]:
for i in range(2017, 2020):
    df = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '_season_schedual.xlsx')
    df = df.reset_index(drop=True)
    df.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '_season_schedual.xlsx')

# test